In [1]:
from algorithms.preprocessing import get_data, word_by_sent, wbys_to_word, word_to_idx, idx_by_sent
# text = get_data('data/EI_original copy.txt')
text = get_data()
wbys = word_by_sent(text)
wordlist = wbys_to_word(wbys)
wtoi = word_to_idx(wordlist)
ibys = idx_by_sent(wbys, wtoi)

In [2]:
from algorithms.textrank import count_window, textrank_keyword, textrank_graph,keywords_to_nodes 
counter = count_window(ibys, 5)
mainkeywords = textrank_keyword(ibys, wordlist)
g_words = textrank_keyword(ibys, wordlist, onlyWords=True)

In [3]:
import algorithms.visualization as vis
import networkx as nx

cnt_draw = vis.counter_draw(counter,g_words)
IG = vis.initialGraph(cnt_draw,g_words)
vis.communityGraph(IG)
subnodes = keywords_to_nodes(mainkeywords)
nx.set_node_attributes(IG, subnodes)

comms = nx.get_node_attributes(IG, 'comm')
weights = nx.get_node_attributes(IG, 'weight')

In [4]:
import pandas as pd

In [5]:
pdc = pd.Series(comms, name='comm')
pdc

bars         0
bar          0
saturated    0
fat          0
energy       1
            ..
watch        7
example      6
granola      6
sugar        6
low          6
Name: comm, Length: 96, dtype: int64

In [6]:
pdw = pd.Series(weights, name='weight')
pdw

bars         52
bar          29
saturated    26
fat          26
energy       19
             ..
watch         3
example       3
granola       3
sugar         3
low           3
Name: weight, Length: 96, dtype: int64

In [7]:
data = pd.concat([pdc, pdw], axis=1)
data

,comm,weight
bars,0,52
bar,0,29
saturated,0,26
fat,0,26
energy,1,19
...,...,...
watch,7,3
example,6,3
granola,6,3
sugar,6,3


In [8]:
data = data.sort_values(by=['comm','weight'], ascending=[True, False])
data

,comm,weight
bars,0,52
bar,0,29
saturated,0,26
fat,0,26
oil,0,10
...,...,...
pick,7,4
general,7,4
equal,7,3
afford,7,3


In [ ]:
result = []
threshold = int(data.iloc[0]['weight'] / 5)
nowComm = -1
size = len(data)
commsize= data.groupby('comm').size()

def makeCircle(idx, cirname):
    if(idx == size) return;
    
    n = data.iloc[idx].name
    c = data.iloc[idx]['comm']
    w = data.iloc[idx]['weight']
    
    if(w < threshold):
        return makeCircle(idx+1, cirname)
    if(nowComm != c):
        result.append({"name": n, "children": []})
        nowComm = c
        result[nowComm]["children"].append(makeCircle(idx+1, n))
    else:
        if(int(commsize[nowComm]* 2/3) <= idx):
            return {"name": n, "size": w}
        else:
            cir = {"name": n, "children":[]}
            cir["children"].append(makeCircle(idx+1, n))
        
            

In [14]:

def makeCircle(newlist):
    if(len(newlist) < 5):
        for w in newlist:
            return {"name": w, "size": weight[w]}
    
    SG = IG.subgraph(newlist)
    vis.communityGraph(SG)
    newcomms = nx.get_node_attributes(SG, 'comm')
    
    newweights = {}
    for k, v in newcomms.items():
        newweights[k] = weights[k]
        
    pdc = pd.Series(newcomms, name='comm')
    pdw = pd.Series(newweights, name='weight')
    df = pd.concat([pdc, pdw], axis=1)
    df = df.sort_values(by=['comm','weight'], ascending=[True, False])
    
    
    for i in range(len(df.groupby('comm').size())):
        if (i == 0):
            bigCircle = {"name": df.iloc[i].name, "children":[]}
            continue
        
        newList = []
        for w in df.index[df['comm'] == i]:
            newList.append(w)
        bigCircle["children"].append(makeCircle(newList))
    
    
for i in range(2):
    newList = []
    for w in data.index[data['comm'] == i]:
        newList.append(w)
    makeCircle(newList)
    
    